In [ ]:
import time
import RPi.GPIO as GPIO

This notebook is basic tutorial for L298N H-bridge ICs demonstrated using RPi.GPIO library, including:
1. 


## 1. L298 Pinout

<img src="Figures/Introduction-to-L298_5.png.webp" alt="L298 Pinout" width="500"/>

The L298 is able to drive two motors, denoted as motor A and motor B. The following pins are used for motor A ("-->" shows how they should be wired up.): 
- Pin 5: INPUT 1 --> Rpi GPIO
- Pin 7: INPUT 2 --> Rpi GPIO
- Pin 2: OUTPUT 1 --> Motor A power
- Pin 3: OUTPUT 2 --> Motor A power
- Pin 6: ENABLE A --> Rpi GPIO
- Pin 1: CURRENT SENSING A --> Ground (This pin is used to measure how much current the motor is drawing, we don't demonstrate this in this tutorial.)

For motor B:
- Pin 10: INPUT 3 --> Rpi GPIO
- Pin 12: INPUT 4 --> Rpi GPIO
- Pin 13: OUTPUT 3 --> Motor B power
- Pin 14: OUTPUT 4 --> Motor B power
- Pin 11: ENABLE B --> Rpi GPIO
- Pin 15: CURRENT SENSING B --> Ground (This pin is used to measure how much current the motor is drawing, we don't demonstrate this in this tutorial.)

The other pins are common for both motors:
- Pin 9: LOGIC SUPPLY VOLTAGE Vss --> 5V
- Pin 8: GND --> Ground
- Pin 4: SUPPLY VOLTAGE Vs --> 12V

## 2. Assigning GPIO

In [ ]:
# Assign pins to GPIOs (For demo only, different GPIOs can be used in practice)
motor_A_in1 = 18
motor_A_in2 = 23
motor_A_en = 13

motor_B_in2 = 24
motor_B_in1 = 25
motor_B_en = 19

# Set GPIO modes
GPIO.setmode(GPIO.BCM)
GPIO.setup(motor_A_in1, GPIO.OUT)
GPIO.setup(motor_A_in2, GPIO.OUT)
GPIO.setup(motor_B_in1, GPIO.OUT)
GPIO.setup(motor_B_in2, GPIO.OUT)
GPIO.setup(motor_A_en, GPIO.OUT)
GPIO.setup(motor_B_en, GPIO.OUT)

## 3. Motor direction control

The direction of motor rotation is controlled by outputing logic High or Low to the two input pins of the L298. For instance:

| INPUT 1 | INPUT 2 | Motor direction |
| :-----: | :-----: | :-------------: |
| Low     | Low     | None            |
| High    | Low     | Clockwise       |
| Low     | High    | Anticlockwise   |
| High    | High    | N/A             |

At the moment, we set both enable pins to High to simply enable full speed for both motors.

In [ ]:
start_time = time.time()

# Rotate both motor L and R anticlockwise for 3 seconds
while time.time() - start_time < 3:
    GPIO.output(motor_A_en, GPIO.HIGH)
    GPIO.output(motor_B_en, GPIO.HIGH)

    GPIO.output(motor_A_in1, GPIO.LOW)
    GPIO.output(motor_A_in2, GPIO.HIGH)

    GPIO.output(motor_B_in1, GPIO.LOW)
    GPIO.output(motor_B_in2, GPIO.HIGH)

GPIO.cleanup()

## 4. Motor speed control using PWM

Output a PWM at enable pins to modulate the speed of the motor.

In [ ]:
# Create PWM instance with a frequency of 100 Hz
pwm_L = GPIO.PWM(motor_A_en, 100)
pwm_R = GPIO.PWM(motor_B_en, 100)

# Start PWM with a duty cycle of 0%
pwm_L.start(0)
pwm_R.start(0)

# Change PWM duty cycle to 20%
pwm_L.ChangeDutyCycle(20)
pwm_R.ChangeDutyCycle(20)

start_time = time.time()

# Rotate both motor L and R anticlockwise for 3 seconds with 20% of the full motor speed
while time.time() - start_time < 3:
    GPIO.output(motor_A_in1, GPIO.LOW)
    GPIO.output(motor_A_in2, GPIO.HIGH)

    GPIO.output(motor_B_in1, GPIO.LOW)
    GPIO.output(motor_B_in2, GPIO.HIGH)

pwm_L.stop()
pwm_R.stop()
GPIO.cleanup()